# Collaborative Filtering 

In [1]:
import pandas as pd

# conda install -c conda-forge scikit-surprise
from surprise import Reader, Dataset, SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise.model_selection import cross_validate, train_test_split
from surprise import accuracy
from surprise.accuracy import rmse

import warnings
warnings.filterwarnings("ignore")

In [198]:
df_posts = pd.read_csv('posts.csv')
df_users = pd.read_csv('users.csv')
df_views = pd.read_csv('views.csv')

df_posts.rename(columns={'_id': 'post_id', ' post_type': 'post_type'}, inplace=True)
df_users.rename(columns={'_id': 'user_id'}, inplace=True)
df_posts.category = df_posts.category.fillna('')

df_merged = pd.merge(df_views, df_users, on='user_id')
df_merged = pd.merge(df_merged, df_posts, on='post_id')

df_merged.drop(columns='timestamp', inplace=True)
df_merged.head()

,user_id,post_id,name,gender,academics,title,category,post_type
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,Niriksha Sharma,female,undergraduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
1,5d7c994d5720533e15c3b1e9,5ec821ddec493f4a2655889e,Varun Chowhan,male,undergraduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
2,5e5af599d701ab08af792b63,5ec821ddec493f4a2655889e,Ilupeju Ayokunnumi,female,graduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
3,5de50d768eab6401affbb135,5ec821ddec493f4a2655889e,thesocialcomment,male,graduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
4,5deeef6142a8854bf6eabab9,5ec821ddec493f4a2655889e,siddharth saxena,male,undergraduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork


In [199]:
print(df_posts.post_type.unique())
print(df_users.gender.unique())
print(df_users.academics.unique())

['blog' 'artwork' 'project' 'skill']
['male' 'female' 'undefined']
['undergraduate' 'graduate' 'undefined']


In [200]:
# assigning weights/ranks to different dependencies
w1 = {'skill': 4.1, 'project': 3, 'artwork': 2.1, 'blog': 0.9}
w2 = {'male': 3.1, 'female': 2.5, 'undefined': 1.5}
w3 = {'graduate': 4, 'undergraduate': 3, 'undefined': 1.5}

df_merged['strength'] = ((df_merged['post_type'].apply(lambda x: w1[x]))/4.1 + (df_merged['gender'].apply(lambda x: w2[x]))/3.1 + (df_merged['academics'].apply(lambda x: w3[x]))/4)/3
df_merged['strength'] = 5*(df_merged['strength'].values/max(df_merged['strength'].values))

df_merged = df_merged[['user_id', 'post_id', 'strength']]
df_merged.head()

,user_id,post_id,strength
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,3.447745
1,5d7c994d5720533e15c3b1e9,5ec821ddec493f4a2655889e,3.770325
2,5e5af599d701ab08af792b63,5ec821ddec493f4a2655889e,3.864411
3,5de50d768eab6401affbb135,5ec821ddec493f4a2655889e,4.186992
4,5deeef6142a8854bf6eabab9,5ec821ddec493f4a2655889e,3.770325


In [201]:
reader = Reader()
data = Dataset.load_from_df(df_merged, reader)
bestAlgo = []

for algo in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    result = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=1)
    temp = pd.DataFrame.from_dict(result).mean(axis=0)
    temp = temp.append(pd.Series([str(algo).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    bestAlgo.append(temp)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.3380  0.3141  0.3303  0.3274  0.3332  0.3286  0.0080  
MAE (testset)     0.2569  0.2423  0.2491  0.2311  0.2547  0.2468  0.0094  
Fit time          0.14    0.16    0.16    0.16    0.15    0.15    0.01    
Test time         0.00    0.00    0.00    0.01    0.00    0.00    0.00    
Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.2584  0.2828  0.2764  0.3009  0.2832  0.2804  0.0137  
MAE (testset)     0.1939  0.2145  0.1966  0.2113  0.2122  0.2057  0.0086  
Fit time          2.30    2.04    2.25    2.33    2.01    2.18    0.13    
Test time         0.03    0.04    0.03    0.03    0.04    0.03    0.00    
Evaluating RMSE, MAE of algorithm SlopeOne on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (

In [202]:
final = pd.DataFrame(bestAlgo).sort_values('test_rmse').set_index('Algorithm')
final

,test_rmse,test_mae,fit_time,test_time
Algorithm,,,,
KNNBaseline,0.225763,0.147803,0.006746,0.009612
KNNWithMeans,0.236272,0.153028,0.002046,0.008061
KNNWithZScore,0.258863,0.171872,0.014621,0.009344
SVDpp,0.280357,0.205703,2.183310,0.033523
SlopeOne,0.287351,0.206045,0.014154,0.020772
SVD,0.328584,0.246824,0.152848,0.004044
BaselineOnly,0.328675,0.244128,0.002991,0.003276
KNNBasic,0.333675,0.246159,0.001924,0.008148
NMF,0.359986,0.274703,0.262041,0.004565


In [203]:
# although all these parameters have some default values. Refer documentation
# other names are pearson_baseline, msd
sim_options = {'name': 'cosine', 'user_based': True, 'shrinkage': 0}    # 'user_based': True means perform user based recommendation, false means do item based recommendation

# Using Alternating Least Squares (ALS)
bsl_optionsA = {'method': 'als', 'reg_u': 15, 'reg_i': 5, 'n_epochs': 20}  # reg_u, reg_i = regularization parameter for users and items
# Using Stochastic Gradient Descent (SGD)
bsl_optionsS = {'method': 'sgd', 'reg': 0.02, 'learning_rate': .00005, 'n_epochs': 20}

algoA = KNNWithMeans(sim_options=sim_options, bsl_options=bsl_optionsA)
algoS = KNNWithMeans(sim_options=sim_options, bsl_options=bsl_optionsS)

print('ALS-------------------------------------------------------------------------------------------------------------')
cross_validate(algoA, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
print('SGD-------------------------------------------------------------------------------------------------------------')
cross_validate(algoS, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

ALS-------------------------------------------------------------------------------------------------------------
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.2274  0.2266  0.2275  0.2668  0.2269  0.2350  0.0159  
MAE (testset)     0.1486  0.1527  0.1564  0.1731  0.1522  0.1566  0.0086  
Fit time          0.01    0.00    0.00    0.01    0.01    0.01    0.00    
Test time         0.01    0.01    0.01    0.01    0.00    0.01    0.00    
SGD------------------------------------------------------------------

{'test_rmse': array([0.22457257, 0.26140969, 0.23522673, 0.23146358, 0.24678294]),
 'test_mae': array([0.14995044, 0.16758806, 0.1579478 , 0.14616916, 0.16777065]),
 'fit_time': (0.005936384201049805,
  0.009357213973999023,
  0.009547948837280273,
  0.008014202117919922,
  0.00709080696105957),
 'test_time': (0.008137941360473633,
  0.00919961929321289,
  0.009109020233154297,
  0.004059314727783203,
  0.008048534393310547)}

In [204]:
train, test = train_test_split(data, test_size=0.2, random_state=200)
algo = KNNWithMeans(algo=KNNWithMeans(sim_options=sim_options, bsl_options=bsl_optionsA))
prediction = algo.fit(train).test(test)
accuracy.rmse(prediction)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.2443


0.24433621848935497

In [205]:
def getU(ruid):
    try:
        return len(train.ur[train.to_inner_uid(ruid)])
    except ValueError: # User id is not a part of trainset
        return 0

def getI(riid):
    try:
        return len(train.ir[train.to_inner_iid(riid)])
    except ValueError: # Item id is not a part of trainset
        return 0

df_new = pd.DataFrame(prediction, columns=['user_id', 'post_id', 'rui', 'est', 'details'])
df_new['no_item_rated_by_user'] = df_new.user_id.apply(getU)
df_new['no_user_rated_item'] = df_new.post_id.apply(getI)
df_new['errors'] = abs(df_new.est - df_new.rui)
df_new.head()

,user_id,post_id,rui,est,details,no_item_rated_by_user,no_user_rated_item,errors
0,5e5855ced701ab08af792b51,5e7bd922cfc8b713f5ac7da9,2.959940,3.711433,"{'was_impossible': True, 'reason': 'User and/o...",0,2,0.751493
1,5ecb979eeaff6b0c3a58a4f0,5eca8fceeaff6b0c3a58a3c0,3.813598,3.835909,"{'actual_k': 6, 'was_impossible': False}",39,7,0.022311
2,5d60098a653a331687083238,5ec278b574f7660d73aa10d5,3.770325,3.654884,"{'actual_k': 5, 'was_impossible': False}",178,5,0.115441
3,5e35a5ed8d344822fed4d13e,5ed0e31a76027d35905cc302,4.230265,4.286960,"{'actual_k': 7, 'was_impossible': False}",16,7,0.056695
4,5e1ef04c2a37d20505da2b8b,5eb1551e10426255a7aaa003,3.447745,3.484953,"{'actual_k': 3, 'was_impossible': False}",46,3,0.037208


In [206]:
bestPred = df_new.sort_values(by='errors')
worstPred = df_new.sort_values(by='errors', ascending=False)

In [207]:
bestPred.head()

,user_id,post_id,rui,est,details,no_item_rated_by_user,no_user_rated_item,errors
278,5d7c994d5720533e15c3b1e9,5eb2cbde10426255a7aaa074,3.770325,3.770325,"{'actual_k': 1, 'was_impossible': False}",75,1,0.0
183,5d610ae1653a331687083239,5eaf8b9310426255a7aa9f7e,5.000000,5.000000,"{'actual_k': 2, 'was_impossible': False}",105,3,0.0
232,5e99b0d4a3258347b42f2bf0,5e9a7e73a3258347b42f2c24,3.770325,3.770325,"{'actual_k': 1, 'was_impossible': False}",3,1,0.0
148,5e7f4ad1a3258347b42f2155,5e897f6ca3258347b42f25cd,3.282520,3.282520,"{'actual_k': 0, 'was_impossible': False}",5,1,0.0
91,5e1ef04c2a37d20505da2b8b,5e2d4d63c85ab714a7da66db,3.447745,3.447745,"{'actual_k': 1, 'was_impossible': False}",46,1,0.0


In [208]:
worstPred.head()

,user_id,post_id,rui,est,details,no_item_rated_by_user,no_user_rated_item,errors
289,5e4ce251f5561b1994c8e40d,5ea7cd9610426255a7aa9bd2,5.000000,3.711433,"{'was_impossible': True, 'reason': 'User and/o...",0,4,1.288567
195,5e5af599d701ab08af792b63,5de8d73249e8203ff9219a74,4.677419,3.711433,"{'was_impossible': True, 'reason': 'User and/o...",50,0,0.965986
45,5d60098a653a331687083238,5ec2d29074f7660d73aa113b,4.583333,3.711433,"{'was_impossible': True, 'reason': 'User and/o...",178,0,0.871900
282,5ea5bf5110426255a7aa9b88,5ea5aacd10426255a7aa9b71,4.230265,3.376606,"{'actual_k': 0, 'was_impossible': False}",1,4,0.853659
96,5e840a75a3258347b42f2437,5e4ed85af5561b1994c8e470,4.552846,3.711433,"{'was_impossible': True, 'reason': 'User and/o...",0,1,0.841412


In [209]:
df_new = pd.merge(df_new, df_posts, on='post_id')
df_new = df_new[['user_id', 'post_id', 'title', 'category', 'post_type', 'rui', 'est', 'errors']]
df_new.head()

,user_id,post_id,title,category,post_type,rui,est,errors
0,5e5855ced701ab08af792b51,5e7bd922cfc8b713f5ac7da9,What sports will look like in the future,Computer Technology|Robotics|Data Science|Info...,blog,2.959940,3.711433,0.751493
1,5ecb979eeaff6b0c3a58a4f0,5eca8fceeaff6b0c3a58a3c0,Zero-Waste Lifestyle,,project,3.813598,3.835909,0.022311
2,5d60098a653a331687083238,5ec278b574f7660d73aa10d5,Rides,Drawings,artwork,3.770325,3.654884,0.115441
3,5e35a5ed8d344822fed4d13e,5ec278b574f7660d73aa10d5,Rides,Drawings,artwork,3.864411,3.881885,0.017473
4,5e35a5ed8d344822fed4d13e,5ed0e31a76027d35905cc302,Designing Cmos circuit from Boolean expression...,,project,4.230265,4.286960,0.056695


In [210]:
df_new[df_new['user_id']==df_new.user_id.value_counts().index[0]].sort_values(by='errors').head()

,user_id,post_id,title,category,post_type,rui,est,errors
117,5d60098a653a331687083238,5eb4fab110426255a7aaa0ed,God Drawing,Drawings,artwork,3.770325,3.781211,0.010886
136,5d60098a653a331687083238,5e52fd0ed701ab08af792a1f,Network Security Threats,Computer Technology|Computer Application,blog,3.282520,3.254238,0.028283
274,5d60098a653a331687083238,5e7df283a3258347b42f2128,screw2,Photography,artwork,3.770325,3.730093,0.040232
230,5d60098a653a331687083238,5e7df068a3258347b42f2125,screw town,Photography,artwork,3.770325,3.730093,0.040232
206,5d60098a653a331687083238,5ecf818376027d35905cbf03,GAN's INTRODUCTION,Computer Technology|Machine Learning,blog,3.282520,3.241290,0.041230


In [211]:
df_test = pd.DataFrame(test, columns=['user_id', 'post_id', 'merged'])
df_test

,user_id,post_id,merged
0,5e5855ced701ab08af792b51,5e7bd922cfc8b713f5ac7da9,2.959940
1,5ecb979eeaff6b0c3a58a4f0,5eca8fceeaff6b0c3a58a3c0,3.813598
2,5d60098a653a331687083238,5ec278b574f7660d73aa10d5,3.770325
3,5e35a5ed8d344822fed4d13e,5ed0e31a76027d35905cc302,4.230265
4,5e1ef04c2a37d20505da2b8b,5eb1551e10426255a7aaa003,3.447745
...,...,...,...
285,5ed237ee76027d35905cc6c5,5e5e3b35fbc8805f69e02c9e,3.699187
286,5df20f1fee4bb5252b4f5351,5e8c2d01a3258347b42f2627,4.136179
287,5d60098a653a331687083238,5e9489e7a3258347b42f2896,3.770325
288,5e5af599d701ab08af792b63,5e9415d2a3258347b42f27f8,3.376606


In [212]:
def recommend(user_id, n=10):
    res = pd.DataFrame(columns=['user_id', 'post_id', 'estimate'])
    for i in df_test.post_id.unique():
        temp = pd.DataFrame([[user_id, i, algo.predict(user_id, i)[3]]], columns=['user_id', 'post_id', 'estimate'])
        res = res.append(temp, ignore_index=True)
    res = pd.merge(res, df_posts, on='post_id')
    return res.sort_values(by='estimate', ascending=False).reset_index(drop=True)[:n]

### Final Recommendation

In [213]:
# recommend(df_test.loc[0,'user_id'])
recommend('5e4ce251f5561b1994c8e40d')

,user_id,post_id,estimate,title,category,post_type
0,5e4ce251f5561b1994c8e40d,5e7bd922cfc8b713f5ac7da9,3.711433,What sports will look like in the future,Computer Technology|Robotics|Data Science|Info...,blog
1,5e4ce251f5561b1994c8e40d,5e948db6a3258347b42f28b2,3.711433,peace,Photography,artwork
2,5e4ce251f5561b1994c8e40d,5e90208ca3258347b42f2730,3.711433,Benefits of Buying Grocery Online in Twin city...,E Commerce|Shopping Platform|Other Online Plat...,blog
3,5e4ce251f5561b1994c8e40d,5ed3476576027d35905cca1d,3.711433,AWS CLI Setup in Mac,Technology,skill
4,5e4ce251f5561b1994c8e40d,5ecf818376027d35905cbf03,3.711433,GAN's INTRODUCTION,Computer Technology|Machine Learning,blog
5,5e4ce251f5561b1994c8e40d,5e830a6ca3258347b42f23f6,3.711433,Dog❤️,Photography,artwork
6,5e4ce251f5561b1994c8e40d,5e8cb88ea3258347b42f267e,3.711433,Palindrome (C# .Net),,project
7,5e4ce251f5561b1994c8e40d,5eb2c11210426255a7aaa052,3.711433,Gangster Style,Drawings,artwork
8,5e4ce251f5561b1994c8e40d,5dada695610ba040fbfdf585,3.711433,Trident,Drawings,artwork
9,5e4ce251f5561b1994c8e40d,5e8d539ba3258347b42f26d8,3.711433,BEING CREATIVE☺☺,Drawings,artwork


### References
<a href='https://surprise.readthedocs.io/en/stable/knn_inspired.html?highlight=knnwithmeans#surprise.prediction_algorithms.knns.KNNWithMeans'>1 relevant surprise documentation</a><br>
<a href='https://surprise.readthedocs.io/en/stable/prediction_algorithms.html#similarity-measures-configuration'>2 relevant surprise documentation</a><br>
<a href='https://surprise.readthedocs.io/en/stable/prediction_algorithms.html#baseline-estimates-configuration'>3 relevant surprise documentation</a><br>